In [20]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [22]:
# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

In [23]:
# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Standardize the feature
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [24]:
class SimpleNN:
    def __init__(self, input_dim, hidden_dim, out_dim):
        self.weights_1 = np.random.randn(input_dim, hidden_dim)
        self.bias_1 = np.zeros((1, hidden_dim))
        self.weights_2 = np.random.randn(hidden_dim, out_dim)
        self.bias_2 = np.zeros((1, out_dim))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, a):
        return a * (1 - a)

    def forward(self, X):
        self.z1 = np.dot(X, self.weights_1) + self.bias_1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.weights_2) + self.bias_2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        self.error = y - output
        delta_output = self.error * self.sigmoid_derivative(output)
        self.d_weights2 = np.dot(self.a1.T, delta_output)
        self.d_bias2 = np.sum(delta_output, axis=0)
        delta_hidden = np.dot(delta_output, self.weights_2.T) * self.sigmoid_derivative(self.a1)
        self.d_weights1 = np.dot(X.T, delta_hidden)
        self.d_bias1 = np.sum(delta_hidden, axis=0)

    def update_weights(self, learning_rate):
        self.weights_1 += self.d_weights1 * learning_rate
        self.bias_1 += self.d_bias1 * learning_rate
        self.weights_2 += self.d_weights2 * learning_rate
        self.bias_2 += self.d_bias2 * learning_rate

    def train(self, X, y, learning_rate, epochs):
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            self.update_weights(learning_rate)

    def predict(self, X):
        output = self.forward(X)
        return (output > 0.5).astype(int)

In [37]:
# Initialize the neural network with the correct number of output classes
nn = SimpleNN(input_dim=4, hidden_dim=5, out_dim=3)
nn.train(X_train, y_train, learning_rate=0.01, epochs=5_000)

In [38]:
# Evaluate the model on the test set
predictions = nn.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

In [36]:
accuracy = np.mean(predicted_classes == true_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 100.00%
